In [1]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

In [4]:
import sqlite3
import pandas as pd

In [2]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


In [20]:
conn = sqlite3.connect('jeju2 copy.db')

In [21]:
query = "SELECT * FROM Review"

In [ ]:
data_df = pd.read_csv('unique_list.csv', encoding='utf-8')

In [ ]:

keywords_data = []

for placeID in data_df['placeID'].unique():
    place_data = data_df[data_df['placeID'] == placeID]
    MCT_NM = place_data['가게명'].iloc[0] 
    reviews = place_data['content'].dropna().tolist()  

    review_text = " ".join(reviews)

    if not review_text:
        print(f"PlaceID: {placeID} | '{MCT_NM}' | No reviews available")
        keywords_data.append({"placeID": placeID, "MCT_NM": MCT_NM, "keywords": "No reviews available"})
        continue

    question = f"'{MCT_NM}'에 대한 리뷰를 바탕으로 관련된 키워드를 3개 이상 추출해줘."

    final_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "너는 음식점 리뷰 관련 키워드를 추출하는 봇이야."),
            ("human", question + f" 리뷰 내용: {{review_text}}"),         ]
    )

    chain = final_prompt | llm
    answer = chain.invoke({"question": question, "review_text": review_text})
    final_answer = answer.content.strip()

    keywords_data.append({"placeID": placeID, "MCT_NM": MCT_NM, "keywords": final_answer})

    print(f"PlaceID: {placeID} | '{MCT_NM}' | Keywords: {final_answer}")

keywords_df = pd.DataFrame(keywords_data)

output_file_path = 'keywords_output.csv'
keywords_df.to_csv(output_file_path, index=False)

print("키워드 리스트가 placeID 별로 CSV 파일에 저장되었습니다.")
print("키워드 리스트가 placeID 별로 CSV 파일에 저장되었습니다.")